<a href="https://colab.research.google.com/github/Daeyoung428/Learning_Python/blob/Master/4th_BackTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance pandas-datareader finance-datareader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 kB 1.5 MB/s eta 0:00:00


In [5]:
# from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

import numpy as np
import pandas as pd

import FinanceDataReader as fdr


In [62]:
def get_Return(code, n):
  df = fdr.DataReader(code, '2018', '2023')

  df = df[['Close']].copy()

  df['MA'] = df.rolling(n).mean().shift(1)

  df['Action'] = np.where(df['Close'] > df['MA'], 'Buy', 'Sell')

  df.iloc[-1, -1] = "Sell"

  cond_Buy = (df['Action'] == "Buy") & (df["Action"].shift(1) == 'Sell')
  cond_Sell = (df['Action'] == "Sell") & (df["Action"].shift(1) == 'Buy')

  df_Buy = df[cond_Buy].reset_index()
  df_Buy.columns = ['날짜', '종가-Buy', '이평값', '액션']
  df_Sell = df[cond_Sell].reset_index()
  df_Sell.columns = ['날짜', '종가-Sell', '이평값', '액션']

  df_result = pd.concat([df_Buy, df_Sell], axis = 1)
  df_result['수익률'] = df_result['종가-Sell'] / df_result['종가-Buy']

  return df_result[['수익률']].cumprod().iloc[-1, -1] - 1

In [90]:
def get_Return_SL(code, short, long):
  df = fdr.DataReader(code, '2018', '2023')

  df = df[['Close']].copy()

  df['MA_1'] = df['Close'].rolling(short).mean().shift(1)
  df['MA_2'] = df['Close'].rolling(long).mean().shift(1)

  df['Action'] = np.where(df['MA_1'] > df['MA_2'], 'Buy', 'Sell')

  df.iloc[-1, -1] = "Sell"

  cond_Buy = (df['Action'] == "Buy") & (df["Action"].shift(1) == 'Sell')
  cond_Sell = (df['Action'] == "Sell") & (df["Action"].shift(1) == 'Buy')

  df_Buy = df[cond_Buy].reset_index()
  df_Buy.columns = ['날짜', '종가-Buy', '이평값1', '이평값2','액션']
  df_Sell = df[cond_Sell].reset_index()
  df_Sell.columns = ['날짜', '종가-Sell', '이평값1', '이평값2', '액션']

  df_result = pd.concat([df_Buy, df_Sell], axis = 1)
  df_result['수익률'] = df_result['종가-Sell'] / df_result['종가-Buy']

  df_final = (df_result[['수익률']].cumprod().tail(1) - 1)*100
  df_final['단기'] = short
  df_final['장기'] = long

  return df_final

In [91]:
get_Return_SL('005930', 5, 120)

,수익률,단기,장기
7,12.691603,5,120
